In [1]:
require 'redis'
require 'date'
redis = Redis.new(host: "redis", port: 6379, db: 0)
redis_second = Redis.new(host: "redis", port: 6379, db: 1)

#<Redis client v4.1.3 for redis://redis:6379/1>

In [2]:
=begin
File.open("partials.array","w"){|f| f.write("") }

i = 0;
redis.keys("{*}").each do |key|
  File.open("partials.array","a"){|f| f.puts( [key,redis.get(key)].to_s ) }
  i = i + 1
  if i % 100000 == 0 then puts "#{Time.now}: #{i} lines written" end
end;   
=end


In [2]:
redis_second.dbsize
#{incremental: true, year: 2017, month: 12, drug: "*", soc: "*" }.to_s

0

In [3]:
pairs = redis.keys({incremental: true, year: 2017, month: 12, drug: "*", soc: "*"}.to_s); nil
pairs = pairs.map{|p| h = (eval p); {drug: h[:drug], soc: h[:soc]} }.uniq
nil

In [31]:
pair = pairs[0]

{:drug=>"ACIDO ALENDRONICO SALE SODICO TRIIDRATO/COLECALCIFEROLO", :soc=>"Nervous system disorders"}

In [7]:
def prr_dynamic_old(pair, redis, vts, vte)
  a,b,c,d = 0,0,0,0
  (vts[0]..vte[0]).each do |year|
      s_month = vts[0] == year ? vts[1] : 1
      e_month = vte[0] == year ? vte[1] : 12
      (s_month..e_month).each do |month|
             ca = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  drug: pair[:drug],
                  soc: pair[:soc]
                 }.to_s)
              a += ca.nil? ? 0.0 : ca.to_f
        
              cb = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  drug: pair[:drug]
                 }.to_s)
              b += cb.nil? ? 0.0 : cb.to_f
        
              cc = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  soc: pair[:soc]
                 }.to_s)
              c += cc.nil? ? 0.0 : cc.to_f
        
              cd = redis.get({
                  incremental: false, 
                  year: year,
                  month: month
                 }.to_s)  
              d += cd.nil? ? 0.0 : cd.to_f
      end  
  end  
  (a/c)/((b - a)/d)
end

:prr_dynamic_old

In [3]:
def prr_dynamic(pair, redis, vts, vte)
  a,b,c,d = 0,0,0,0
  (vts[0]..vte[0]).each do |year|
      s_month = vts[0] == year ? vts[1] : 1
      e_month = vte[0] == year ? vte[1] : 12
      (s_month..e_month).each do |month|
             ca = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  drug: pair[:drug],
                  soc: pair[:soc]
                 }.to_s)
              a += ca.nil? ? 0.0 : ca.to_f
        
              cb = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  drug: pair[:drug]
                 }.to_s)
              b += cb.nil? ? 0.0 : cb.to_f
        
              cc = redis.get({
                  incremental: false, 
                  year: year,
                  month: month,
                  soc: pair[:soc]
                 }.to_s)
              c += cc.nil? ? 0.0 : cc.to_f
        
              cd = redis.get({
                  incremental: false, 
                  year: year,
                  month: month
                 }.to_s)  
              d += cd.nil? ? 0.0 : cd.to_f
      end  
  end  
  s = Math.sqrt((1/a) + (1/c) + (-1/(a +b)) + (-1/(c+d)))
  prr = (a/c)/((b - a)/d)
  {
    a: a,
    b: b,
    c: c,
    d: d,
    s: s,
    prr: prr,
    count_cases: a,
    confidence: [prr/ Math.exp(1.96 * s), prr * Math.exp(1.96 * s)],
    chi_square: ((a*d - b*c)**2)*(a+b+c+d)/((a+b)*(c+d)*(a+c)*(b+d))
    }
end

:prr_dynamic

In [8]:
pair = pairs.sample
prr_dynamic(pair, redis, [2008,1], [2016,8])
pair

{:drug=>"ZOLPIDEM TARTRATO", :soc=>"Eye disorders"}

In [25]:
#pairs.each do |p|
#  redis.set(p, prr(p,redis))
#end  

In [4]:
#static case: lenght 1, search the next year/month
#need a dynamic case where lengh is replaced by the end of intervall specified in file
def vt_next_month(vt, length)
  years = (length / 12) + vt[0]
  r = [vt[0], nil]
  if vt[1] + (length % 12) > 12
    r[0] = years + 1
    r[1] = vt[1] + (length % 12) - 12
  else 
    r[0] = years 
    r[1] = vt[1] + (length % 12)
  end  
  r
end
  
def prr_split(pair, redis, length, min_vt = [2008,1], max_vt = [2017,12])
  vts, r = min_vt, []
  while true do
   vte = vt_next_month(vts, length)
   if vte[0] > max_vt[0] || (vte[0] == max_vt[0] && vte[1] > max_vt[1]) then break end
   r.push({
     drug: pair[:drug],
     soc: pair[:soc],
     [vts,vte] => prr_dynamic(pair, redis, vts, vte)
     })  
   vts = vt_next_month(vts, 1)  
  end
  r
end  

:prr_split

In [13]:
prr_split(pair, redis, 1)

[{:drug=>"ACIDO URSODESOSSICOLICO", :soc=>"Vascular disorders", [[2008, 1], [2008, 2]]=>{:a=>0.0, :b=>6.0, :c=>295.0, :d=>7384.0, :s=>Infinity, :prr=>0.0, :count_cases=>0.0, :confidence=>[0.0, NaN], :chi_square=>0.23969999899555594}}, {:drug=>"ACIDO URSODESOSSICOLICO", :soc=>"Vascular disorders", [[2008, 2], [2008, 3]]=>{:a=>0.0, :b=>8.0, :c=>275.0, :d=>7630.0, :s=>Infinity, :prr=>0.0, :count_cases=>0.0, :confidence=>[0.0, NaN], :chi_square=>0.2883250116473791}}, {:drug=>"ACIDO URSODESOSSICOLICO", :soc=>"Vascular disorders", [[2008, 3], [2008, 4]]=>{:a=>0.0, :b=>5.0, :c=>258.0, :d=>8063.0, :s=>Infinity, :prr=>0.0, :count_cases=>0.0, :confidence=>[0.0, NaN], :chi_square=>0.15998700386934586}}, {:drug=>"ACIDO URSODESOSSICOLICO", :soc=>"Vascular disorders", [[2008, 4], [2008, 5]]=>{:a=>0.0, :b=>4.0, :c=>300.0, :d=>9070.0, :s=>Infinity, :prr=>0.0, :count_cases=>0.0, :confidence=>[0.0, NaN], :chi_square=>0.1323024325779568}}, {:drug=>"ACIDO URSODESOSSICOLICO", :soc=>"Vascular disorders", [[

In [69]:
#COMPUTE dynamic (year only)
START_YEAR=2007 #2008-1    for 5y version 2012

File.readlines('split_yearly.txt').each do |line|
  array=line.strip.split("\t")
  pair = { :drug => array[0], :soc => array[1] } 
  sp=array[2].split(",").map(&:to_i)
  sp.each_with_index{ |x,index| 
    if index != sp.size-1 
      if (x == 0 && sp[index+1] !=1)
        x=1
      end
      if (x!=0)
        #puts [x+START_YEAR,1] #-> vts
        #puts [sp[index+1]+START_YEAR-1,12] #-> vte (closed intervall in prr_dynamic)
        res=prr_dynamic(pair,redis,[x+START_YEAR,1],[sp[index+1]+START_YEAR-1,12])
        #drug,soc,vts,vte,prr, confidence-,confidence+,chi, count
        File.write("result_dyn_year.txt", pair[:drug]+"\t"+pair[:soc]+"\t"+[x+START_YEAR,1].to_s+"\t"+[sp[index+1]+START_YEAR-1,12].to_s+"\t"+res[:prr].to_s+"\t"+res[:confidence][0].to_s+"\t"+res[:confidence][1].to_s+"\t"+res[:chi_square].to_s+"\t"+res[:count_cases].to_i.to_s+"\n", mode: "a")
      end
    end
    }
end

<main>:1: warning: already initialized constant START_YEAR
<main>:1: warning: previous definition of START_YEAR was here


["BROTIZOLAM\tCardiac disorders\t0,1,3,4,8,10,11\n", "DESTROMETORFANO\tEar and labyrinth disorders\t0,1,6,7,11\n", "CLOPIDOGREL BROMIDRATO MONOIDRATO\tReproductive system and breast disorders\t0,1,11\n", "MIRABEGRON\tNeoplasms benign, malignant and unspecified (incl cysts and polyps)\t0,1,11\n", "AMPICILLINA\tRespiratory, thoracic and mediastinal disorders\t0,1,2,3,8,9,11\n", "DOMPERIDONE\tSkin and subcutaneous tissue disorders\t0,1,2,3,4,11\n", "CLOBAZAM\tReproductive system and breast disorders\t0,1,11\n", "DOXORUBICINA CLORIDRATO\tPregnancy, puerperium and perinatal conditions\t0,1,5,6,11\n", "IRINOTECAN\tInjury, poisoning and procedural complications\t0,1,10,11\n", "LATTULOSIO/PARAFFINA LIQUIDA/PARAFFINA BIANCA SOFFICE\tPsychiatric disorders\t0,1,11\n", "PSEUDOEFEDRINA/TRIPROLIDINA\tImmune system disorders\t0,1,11\n", "IMIPRAMINA CLORIDRATO\tNervous system disorders\t0,1,6,7,11\n", "LERCANIDIPINA\tInvestigations\t0,1,9,10,11\n", "DIMETINDENE MALEATO\tSkin and subcutaneous tissue di

In [5]:
#COMPUTE dynamic (month only)
START_MONTH=[2008,1] #different from yearly version (not -1) for 5y version [2013,1]

def start_month(month)
  monthToAdd=(month-1)%12
  year=(month-1)/12.to_i
  [START_MONTH[0]+year, START_MONTH[1]+monthToAdd]
end

def end_month(month)
  start_month(month-1)
end
  
File.readlines('split_DinEps-v4_chi5_ci5.txt').each do |line|
  array=line.strip.split("\t")
  pair = { :drug => array[0], :soc => array[1] } 
  sp=array[2].split(",").map(&:to_i)
  sp.each_with_index{ |x,index| 
    if index != sp.size-1 
      if (x == 0 && sp[index+1] !=1)
        x=1
      end
      if (x != 0)
      #puts "Start "+start_month(x).to_s# -> vts
      #puts "End "+end_month(sp[index+1]).to_s #-> vte (closed intervall in prr_dynamic)
      res=prr_dynamic(pair,redis,start_month(x),end_month(sp[index+1]))
      #drug,soc,vts,vte,prr, confidence-,confidence+,chi, count
      File.write("result_split_DinEps-v4_chi5_ci5.txt", pair[:drug]+"\t"+pair[:soc]+"\t"+start_month(x).to_s+"\t"+end_month(sp[index+1]).to_s+"\t"+res[:prr].to_s+"\t"+res[:confidence][0].to_s+"\t"+res[:confidence][1].to_s+"\t"+res[:chi_square].to_s+"\t"+res[:count_cases].to_i.to_s+"\n", mode: "a")
      end
    end
    }
end

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [13]:
#static prr monthly ESTREMI INCLUSI
def prr_monthly(pair, redis, vts, vte)
  current = vts
  r=[]
  while current != vte
    r.push( prr_dynamic(pair, redis, current, current).merge({month: [current[0],current[1]]}))
    if current[1] == 12 
      current[1]  = 1
      current[0] += 1
    else  
      current[1] += 1
    end  
  end  
  r.push(prr_dynamic(pair, redis, current, current).merge({ month: [current[0],current[1]]})) 
end 



:prr_monthly

In [55]:
redis.keys({drug: "*", soc: "*"}.to_s).each do |key_string|
   pair = eval key_string
   res=prr_monthly(pair,redis,[2008,1],[2017,12])
   res.each{|val| 
     File.write("result_stc_month.txt", pair[:drug].to_s+"\t"+pair[:soc].to_s+"\t"+val[:month].to_s+"\t"+val[:month].to_s+"\t"+val[:prr].to_s+"\t"+val[:confidence][0].to_s+"\t"+val[:confidence][1].to_s+"\t"+val[:chi_square].to_s+"\t"+val[:count_cases].to_i.to_s+"\n", mode: "a")
     }
end

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [166]:
#static prr yearly     DESTRO ESCLUSO
def prr_yearly(pair, redis, vts, vte)
  current = vts
  next_current = Array.new(2,0)
  r = []
  while current[0] < vte[0]
    if (current[1] == 1)
      next_current[1] = 12
      next_current[0] = current[0]
    else
      next_current[1] = current[1] - 1
      next_current[0] = current[0] + 1
    end      
    r.push( prr_dynamic(pair, redis, current, next_current).merge({vts: [current[0],current[1]]}).merge({vte: [next_current[0],next_current[1]]}))
    current[0] += 1
  end  
  r
end


:prr_yearly

In [169]:
redis.keys({drug: "*", soc: "*"}.to_s).each do |key_string|
   pair = eval key_string
   res=prr_yearly(pair,redis,[2008,1],[2018,1])
   res.each{|val| 
     File.write("result_stc_year.txt", pair[:drug].to_s+"\t"+pair[:soc].to_s+"\t"+val[:vts].to_s+"\t"+val[:vte].to_s+"\t"+val[:prr].to_s+"\t"+val[:confidence][0].to_s+"\t"+val[:confidence][1].to_s+"\t"+val[:chi_square].to_s+"\t"+val[:count_cases].to_i.to_s+"\n", mode: "a")
     }
end

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [44]:
#PRR static quarterly
#ESTREMI INCLUSI
def prr_quarterly(pair, redis, vts, vte)
  current = vts
  next_current = Array.new(2,0)
  next_current[0]=current[0]
  next_current[1]=current[1]+2
  r=[]
  while next_current != vte
    r.push( prr_dynamic(pair, redis, current,next_current).merge({vts: [current[0],current[1]]}).merge({vte: [next_current[0],next_current[1]]}))
    if current[1] == 10 
      current[1]  = 1
      current[0] += 1
    else  
      current[1] += 3
    end
    next_current[0]=current[0]
    next_current[1]=current[1]+2
  end  
    r.push( prr_dynamic(pair, redis, current,next_current).merge({vts: [current[0],current[1]]}).merge({vte: [next_current[0],next_current[1]]}))
end 



:prr_quarterly

In [45]:
redis.keys({drug: "*", soc: "*"}.to_s).each do |key_string|
   pair = eval key_string
   res=prr_quarterly(pair,redis,[2008,1],[2017,12])
   res.each{|val| 
     File.write("result_stc_quarterly.txt", pair[:drug].to_s+"\t"+pair[:soc].to_s+"\t"+val[:vts].to_s+"\t"+val[:vte].to_s+"\t"+val[:prr].to_s+"\t"+val[:confidence][0].to_s+"\t"+val[:confidence][1].to_s+"\t"+val[:chi_square].to_s+"\t"+val[:count_cases].to_i.to_s+"\n", mode: "a")
    }
end

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [1]:
a=[0,0,1,2,0,0]

[0, 0, 1, 2, 0, 0]

In [2]:
require 'descriptive_statistics'

a.mean


0.5

In [3]:
a.standard_deviation/a.mean

1.5275252316519468

In [4]:
b=[1,1,1,1,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,1,1,1,1,1]
b.standard_deviation/b.mean
b.size

27

In [5]:
eps = (a.standard_deviation/a.mean)/(b.standard_deviation/b.mean)

1.2926345016292344

In [6]:
class Array
  def minimal_intervals(min_cases, zeroes)   
  intervals = []
  (0..size - 1).each do |i|
    found = nil
    if i == 0 || self[i - 1] == 0
      (i..size - 1).each{|j|
        if self[i..j].select{|item| item == 0}.size > zeroes then break end
        if self[i..j].sum == min_cases
          if j == size - 1 || self[j + 1] == 0
            found = j
            break
          end
        else  
          if self[i..j].sum > min_cases then break end
        end  
      }
    end
    if found
      intervals.push [i, found]
    end  
  end
  intervals    
  end   
end  

:minimal_intervals

In [7]:
b.minimal_intervals(3,0)

[[11, 12]]

In [8]:
b.each_with_index.to_a

[[1, 0], [1, 1], [1, 2], [1, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [1, 11], [2, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [1, 21], [1, 22], [1, 23], [1, 24], [1, 25], [1, 26]]